In [26]:
import pandas as pd
import numpy as np
import os

In [2]:
import pylab as pl
%matplotlib inline

/home/cmp/anaconda2/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


In [76]:
names=['OptLevel','N','run','usec']

In [77]:
gb = pd.read_csv("g_laptop", header=None, delimiter=" ", names=names).drop("usec", axis=1)

In [78]:
for f in os.listdir('.'):
    if f[:2]=="g_":
        g = pd.read_csv(f, header=None, delimiter=" ", names=names[:-1] + [f])
        gb = gb.merge(g, on=names[:-1])

In [79]:
gb.groupby(names[:2]).aggregate(np.mean).drop("run", axis=1)

g_crackle4  g_crunchy5  g_snappy5  g_energon2  g_crunchy4  \
OptLevel N                                                                   
0        100        11834.8      7584.2     8582.9      9019.6      7584.9   
         1000       36417.1     37406.3    26090.8     39921.8     28774.5   
         10000     284780.4    338263.0   195807.6    333501.2    253020.3   
         100000   2734191.7   3312990.5  1874765.8   3234480.2   2512906.1   
1        100         9393.4      6286.9     6209.6      6710.2      4950.2   
         1000       18728.7     18280.5    16503.5     18895.4     16041.3   
         10000      93784.1    144255.3    69542.8    131070.6    120476.9   
         100000    853748.6   1398125.6   629829.1   1236992.1   1213986.6   
2        100         8751.0      5750.3    11688.9      6636.0      5131.8   
         1000       17210.8     18748.5    31300.7     18661.1     15936.0   
         10000      94955.1    144987.7    73922.2    130193.8    119737.6   
         100000    853776.5   1403382.5   629519.6   1230058.0   1204885.6   
3        100         9593.2      4947.2     5931.7      6318.1      4931.2   
         1000       16418.8     13069.7    13465.2     15546.5     12130.0   
         10000      67361.8     89797.6    53636.3     96580.9     81699.7   
         100000    592299.6    858324.8   460541.5    904908.8    785853.1   

                  g_laptop  g_crunchy6  g_brawler  
OptLevel N                                         
0        100        8327.0      7707.8     6684.5  
         1000      34952.4     37679.7    31264.9  
         10000    297923.8    338332.5   271025.7  
         100000  2760091.5   3369284.1  2679352.8  
1        100       12253.1      6195.0     5219.7  
         1000      22717.9     19282.1    15417.5  
         10000    151109.3    143759.7   117451.3  
         100000  1423747.9   1397429.9  1177645.4  
2        100        7504.3      8495.8     4930.6  
         1000      25871.1     20230.8    15580.2  
         10000    148082.1    144898.2   119481.7  
         100000  1397991.8   1421432.4  1181133.0  
3        100        6342.7      6251.1     4428.9  
         1000      17455.6     14282.7    11922.4  
         10000     93648.4     96598.2    81606.8  
         100000   899483.3    856562.6   812209.9

In [80]:
gb = pd.read_csv("j_laptop", header=None, delimiter=" ", names=names).drop("usec", axis=1)

In [81]:
for f in os.listdir('.'):
    if f[:2]=="j_":
        g = pd.read_csv(f, header=None, delimiter=" ", names=names[:-1] + [f])
        gb = gb.merge(g, on=names[:-1])

In [82]:
gb.groupby(names[:2]).aggregate(np.mean).drop("run", axis=1)

j_laptop  j_energon2  j_crunchy6  j_crunchy4  j_snappy5  \
OptLevel N                                                                  
0        100       14711.5      9961.7     12966.6      7629.2    17034.2   
         1000      35800.9     44694.5     64213.5     35409.7    30607.9   
         10000    309888.8    385208.0    411732.3    318202.8   220137.6   
         100000  2949398.0   3774649.1   4024117.7   3199770.4  2086347.2   
1        100       11613.5      8195.7      6310.8      4908.2     6442.4   
         1000      22723.9     13346.2     12552.2     11280.7    10048.2   
         10000    136098.3     67334.4     57074.3     60342.4    36657.0   
         100000  1324188.8   1754490.7    741629.1    696851.4   315704.5   
2        100        9657.7      6388.4      6818.4      4867.4     7826.2   
         1000      26140.6     12706.2     10893.5     10675.0    11871.7   
         10000    140680.9     65871.7     54964.3     57257.6    36518.5   
         100000  1316717.0   1748924.1    604897.7    698458.2   309747.0   
3        100        6492.3      6502.4      5085.5      4840.1    12133.2   
         1000      15348.0     12495.9     10188.3      9928.4    19240.3   
         10000     78547.8     64924.1     53739.5     57315.6    42220.0   
         100000   782105.7   1737034.6    577561.5    713851.3   297823.2   

                 j_crunchy5  j_brawler  j_crackle4  
OptLevel N                                          
0        100         8604.8     7493.4     12707.2  
         1000       42962.7    37425.8     40305.1  
         10000     392381.8   331874.6    325613.3  
         100000   3897640.7  3504816.2   3137678.2  
1        100         4823.8     4466.4      9592.3  
         1000        9875.3    10184.6     14200.4  
         10000      53120.1    74898.3     49359.8  
         100000    587405.3  1543390.2    443603.6  
2        100         4829.3     5008.3      9629.0  
         1000        9988.2    10305.2     15401.7  
         10000      53762.6    74948.3     56060.0  
         100000    605007.5  1530518.2    474910.0  
3        100         5598.9     4570.9      9505.8  
         1000        9287.3     9843.8     13354.5  
         10000      52515.0    74612.4     43803.1  
         100000    597635.1  1536742.5    372377.8